In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.95'
os.environ['JAX_ENABLE_X64'] = 'False'

In [3]:
from scipy import stats
import numpy as np

from synthetic_utils import matrix1, matrix2, matrix3, matrix_gersh
from synthetic_utils import gen_synthetic_dataset, bx_ones_sol
from synthetic_utils import load_synthetic_dataset

In [20]:
size = 1e4
density = .05
alpha = 1

A1 = matrix_gersh(size, density, alpha, seed=42)
A2 = matrix_gersh(size, density, alpha, seed=42)

In [21]:
A1.indices

array([   0,    1,   11, ..., 9950, 9977, 9999], dtype=int32)

In [22]:
A2.indices

array([   0,    1,   11, ..., 9950, 9977, 9999], dtype=int32)

In [24]:
np.allclose(A1.data, A2.data)

False

In [3]:
import jax.numpy as jnp
jnp.ones(2)

2024-11-24 16:47:27.449659: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


Array([1., 1.], dtype=float32)

In [4]:
folder_path = '/mnt/local/data/vtrifonov/prec-learning-Notay-loss/synthetic_datasets'

In [100]:
# Matrix Gersh dataset

file_path = os.path.join(folder_path, 'matrixGersh_sol_ones_nnz0.5%.npz')
N = 3
size = 1e3
density = .05
alpha = 0.6
lhs_func = matrix_gersh
bx_func = bx_ones_sol
sol_func = np.ones
rhs_func = lambda A, x: A @ x
lhs_distr = stats.uniform(loc=-1, scale=2) # stats.norm(loc=0, scale=1)

gen_synthetic_dataset(file_path, N, size, density, alpha,
                      bx_func, sol_func, rhs_func, lhs_func, 
                      lhs_distr=lhs_distr,
                      prec_type='ic(0)')

Started
0! 
Current len: 0. Matrix is not positive definite
0! 
Current len: 0. Matrix is not positive definite
0! 
Current len: 0. Matrix is not positive definite
0! 
Current len: 0. Matrix is not positive definite
0! 1! 2! 
All systems and precs are created
Condition numbers are calculated
Saved


In [101]:
abc = np.load(file_path, allow_pickle=True)

In [102]:
list(abc.keys())

['A', 'L', 'b', 'x', 'nnz_A', 'nnz_L', 'cond_A', 'max_len', 'min_len']

In [103]:
abc['cond_A']

array([426.0, 1470.0, 4216.0], dtype=object)

In [104]:
abc['nnz_A']

array([5.082, 5.082, 5.082], dtype=object)

In [2]:
import numpy as np
import jax.numpy as jnp
from scipy import sparse, stats

In [3]:
def matrix1(size, density, seed=42, sample_distr=stats.uniform(loc=-1, scale=2)):
    size = int(size)
    A = sparse.random(size, size, density=density, format='csr',
                      random_state=np.random.default_rng(seed),
                      data_rvs=sample_distr.rvs)
    return (sparse.eye(size) + A) @ (sparse.eye(size) + A.T)

def dataset_matrix1(name, N, size, density, path):
    A_list = []
    for _ in range(N):
        A_list.append(matrix1(size, density, sample_distr=stats.norm(loc=0, scale=1)))
    nnz = A_n.nnz * 100 / (n * n)
    cond_n = jnp.linalg.cond(jnp.asarray(A_n.todense()))
    print(f'Size = {n:.0f}, nnz = {nnz:.1f}%, cond = {cond_n:.1f}')

In [4]:
size = 1e4
density = 0.001
N_train = 1000
N_test = 200

In [6]:
# A_train

for _ in range(3):
    A_n = matrix1(size, density, sample_distr=stats.norm(loc=0, scale=1))
    nnz = A_n.nnz * 100 / (size * size)
    cond_n = jnp.linalg.cond(jnp.asarray(A_n.todense()))
    print(f'Size = {size:.0f}, nnz = {nnz:.1f}%, cond = {cond_n:.1f}')

Size = 10000, nnz = 1.2%, cond = 548952704.0
Size = 10000, nnz = 1.2%, cond = 2165645312.0
Size = 10000, nnz = 1.2%, cond = 581705600.0


In [7]:
check = np.linalg.cholesky(A_n.todense())

In [9]:
import ilupp
import pymatting

L_ilupp = ilupp.ichol0(A_n)
L_pymat = pymatting.ichol(A_n, discard_threshold=1e-2).L

print('NaNs')
print('  ilupp:', np.any(np.isnan(L_ilupp.todense())))
print('  pymatting:', np.any(np.isnan(L_pymat.todense())))
print()

print('Min abs on diagonal')
print('  ilupp:', np.min(np.abs(L_ilupp.diagonal())))
print('  pymatting:', np.min(np.abs(L_pymat.diagonal())))

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-02
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-02
    shift = 1.000000e-04
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-02
    shift = 1.000000e-03
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-02
    shif

In [ ]:
# for n in [1e3, 1e4, 1e5]:
#     A_n = matrix_gesh(n, sample_distr=stats.norm(loc=0, scale=1))
#     nnz = A_n.nnz * 100 / (n * n)
#     cond_n = jnp.linalg.cond(jnp.asarray(A_n.todense()))
#     print(f'Size = {n:.0f}, nnz = {nnz:.1f}%, cond = {cond_n:.1f}')

In [47]:
matrix_gesh(1000).nnz

50820

In [12]:
np.ones([10, 7]).sum(1).shape

(10,)

In [20]:
a = np.random.default_rng()

In [ ]:
a.uniform()